In [1]:
import numpy as np
import pandas as pd
import pickle

# 1. Data
- Load data from pickle file
- split the data into independent and dependent
- split to train and test set

In [2]:
data = pickle.load(open('data_face_features.pickle',mode='rb'))

In [3]:
X = np.array(data['data']) # indendepent variable
y = np.array(data['label']) # dependent variable

In [4]:
X.shape , y.shape

((394, 1, 128), (394,))

In [5]:
X = X.reshape(-1,128)
X.shape

(394, 128)

In [6]:
# split the data into train and test
from sklearn.model_selection import train_test_split

In [7]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [8]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((315, 128), (79, 128), (315,), (79,))

# 2. Train Machine Learning

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

### Logistic Regression

In [10]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) # training logistic regression

LogisticRegression()

In [11]:
def get_report(model, x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [12]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.90
Accuracy Test = 0.84
F1 Score Train = 0.91
F1 Score Test = 0.83


### Support Vector Machines

In [13]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [14]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.95
Accuracy Test = 0.90
F1 Score Train = 0.95
F1 Score Test = 0.90


### Random Forest

In [15]:
model_rf = RandomForestClassifier(n_estimators=10,)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [16]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 0.99
Accuracy Test = 0.82
F1 Score Train = 0.99
F1 Score Test = 0.82


## Voting Classifier

In [17]:
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [18]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [19]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.97
Accuracy Test = 0.86
F1 Score Train = 0.97
F1 Score Test = 0.86


# 3. Parameter Tuning

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
model_grid = GridSearchCV(model_voting,
             param_grid={
               'svm__C':[3,5,7,10],
               'svm__gamma':[0.1,0.3,0.5],
               'rf__n_estimators':[5,10,20],
               'rf__max_depth':[3,5,7],
               'voting':['soft']
             },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [22]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[

[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5

[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7

[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=10, svm

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=   0.1s


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft']},
             scoring='accuracy', verbose=2)

In [23]:
model_best_estimator = model_grid.best_estimator_

In [24]:
model_grid.best_score_

0.8984126984126984

# 4. Save Model

In [25]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))